# **Kitchen Object Classification**

In [1]:
# Fundamental classes
import numpy as np
import pandas as pd
import tensorflow as tf
import os

# Image related
import cv2
from PIL import Image

#Performance Plot
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For the model and it's training
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, Add
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Time
import time
import datetime

# Preprocessing

**Region extraction from images**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Llegir el fitxer d'anotacions
annotations_path = '/content/drive/MyDrive/TFG/kitchen-obj-classification/_annotations_regions.csv'
annotations_df = pd.read_csv(annotations_path)

# Definir el camí de les imatges
IMAGES_PATH = '/content/drive/MyDrive/TFG/kitchen-obj-seg/train-dataset/images/'

# Funció per carregar i extreure les regions anotades per cada classe
def load_annotated_regions_for_class(annotations_df, class_name, images_path, target_size=(150, 150)):
    images = []
    labels = []
    class_df = annotations_df[annotations_df['class'] == class_name]
    for _, row in class_df.iterrows():
        img_path = os.path.join(images_path, row['filename'])
        image = load_img(img_path, color_mode='grayscale')
        image = img_to_array(image)
        x_min = int(row['xmin'])
        y_min = int(row['ymin'])
        x_max = int(row['xmax'])
        y_max = int(row['ymax'])
        region = image[y_min:y_max, x_min:x_max]
        region = cv2.resize(region, target_size)
        region = region / 255.0  # Normalitzar
        images.append(region)
        labels.append(row['labels'])
    return np.array(images), np.array(labels)

# Carregar les dades per cada classe
classes = ['sink', 'floor', 'cabinet', 'counter', 'tiles']
data = {}

for class_name in classes:
    x, y = load_annotated_regions_for_class(annotations_df, class_name, IMAGES_PATH)
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    data[class_name] = (x_train, x_val, y_train, y_val)

# Mostrar un exemple de dades carregades
for class_name, (x_train, x_val, y_train, y_val) in data.items():
    print(f"Class: {class_name}, Train samples: {x_train.shape[0]}, Validation samples: {x_val.shape[0]}")


Class: sink, Train samples: 106, Validation samples: 27
Class: floor, Train samples: 114, Validation samples: 29
Class: cabinet, Train samples: 190, Validation samples: 48
Class: counter, Train samples: 162, Validation samples: 41
Class: tiles, Train samples: 182, Validation samples: 46


# Training

**Simple CNN**

Model

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def create_simple_cnn(input_shape=(150, 150, 1)):
    model = models.Sequential([
        layers.Conv2D(32, (5, 5), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (5, 5), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(rate=0.25),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(rate=0.25),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(rate=0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

create_simple_cnn().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 146, 146, 32)      832       
                                                                 
 batch_normalization (Batch  (None, 146, 146, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 142, 142, 32)      25632     
                                                                 
 batch_normalization_1 (Bat  (None, 142, 142, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 71, 71, 32)        0         
 D)                                                              
                                                        

Training

In [7]:
# Train a simple CNN model for each class
trained_models_simple_cnn = {}

for class_name, (x_train, x_val, y_train, y_val) in data.items():
    print(f"Training simple CNN model for class: {class_name}")

    model = create_simple_cnn(input_shape=(150, 150, 1))

    # Define callbacks
    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ModelCheckpoint(f'/content/drive/MyDrive/TFG/kitchen-obj-classification/models/simple_cnn_model_{class_name}.h5', save_best_only=True)
    ]

    # Train the model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=50,
        batch_size=16,
        callbacks=callbacks
    )

    trained_models_simple_cnn[class_name] = model

    print(f"Simple CNN model for class {class_name} trained and saved.")

Training simple CNN model for class: sink
Epoch 1/50
7/7 [==============================] - ETA: 0s - loss: 21.5155 - accuracy: 0.4811

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 14s 740ms/step - loss: 21.5155 - accuracy: 0.4811 - val_loss: 1.0130 - val_accuracy: 0.4815
Epoch 2/50
7/7 [==============================] - 0s 73ms/step - loss: 9.1678 - accuracy: 0.6604 - val_loss: 4.4684 - val_accuracy: 0.4815
Epoch 3/50
7/7 [==============================] - 0s 55ms/step - loss: 3.2074 - accuracy: 0.8019 - val_loss: 3.7786 - val_accuracy: 0.4815
Epoch 4/50
7/7 [==============================] - 0s 54ms/step - loss: 4.1326 - accuracy: 0.8585 - val_loss: 5.6160 - val_accuracy: 0.4444
Epoch 5/50
7/7 [==============================] - 0s 58ms/step - loss: 2.4855 - accuracy: 0.8491 - val_loss: 3.3039 - val_accuracy: 0.4815
Epoch 6/50
7/7 [==============================] - 0s 64ms/step - loss: 2.0137 - accuracy: 0.8962 - val_loss: 3.1454 - val_accuracy: 0.4815
Epoch 7/50
7/7 [==============================] - 0s 58ms/step - loss: 1.5509 - accuracy: 0.9151 - val_loss: 11.5360 - val_accuracy: 0.4815
Epoch 8/50
7/7 [==================

**VGG**

Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_vgg_model(input_shape=(150, 150, 1)):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

create_vgg_model().summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 150, 150, 64)      640       
                                                                 
 conv2d_25 (Conv2D)          (None, 150, 150, 64)      36928     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 75, 75, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 75, 75, 128)       73856     
                                                                 
 conv2d_27 (Conv2D)          (None, 75, 75, 128)       147584    
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 37, 37, 128)       0         
 ng2D)                                                

Training

In [9]:
# Train a VGG-like model for each class
trained_models_vgg = {}

for class_name, (x_train, x_val, y_train, y_val) in data.items():
    print(f"Training VGG model for class: {class_name}")

    model = create_vgg_model(input_shape=(150, 150, 1))

    # Define callbacks
    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True),
        ModelCheckpoint(f'/content/drive/MyDrive/TFG/kitchen-obj-classification/models/vgg_model_{class_name}.h5', save_best_only=True)
    ]

    # Train the model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=50,
        batch_size=16,
        callbacks=callbacks
    )

    trained_models_vgg[class_name] = model

    print(f"VGG model for class {class_name} trained and saved.")

Training VGG model for class: sink
Epoch 1/50
7/7 [==============================] - 32s 4s/step - loss: 0.8521 - accuracy: 0.5377 - val_loss: 0.6943 - val_accuracy: 0.4815
Epoch 2/50
7/7 [==============================] - 1s 93ms/step - loss: 0.6943 - accuracy: 0.5189 - val_loss: 0.6944 - val_accuracy: 0.4815
Epoch 3/50
7/7 [==============================] - 5s 751ms/step - loss: 0.7006 - accuracy: 0.4906 - val_loss: 0.6925 - val_accuracy: 0.5185
Epoch 4/50
7/7 [==============================] - 5s 833ms/step - loss: 0.6911 - accuracy: 0.5472 - val_loss: 0.6924 - val_accuracy: 0.5185
Epoch 5/50
7/7 [==============================] - 1s 90ms/step - loss: 0.6949 - accuracy: 0.4906 - val_loss: 0.6932 - val_accuracy: 0.5185
Epoch 6/50
7/7 [==============================] - 1s 86ms/step - loss: 0.6943 - accuracy: 0.5283 - val_loss: 0.6928 - val_accuracy: 0.5185
Epoch 7/50
7/7 [==============================] - 1s 86ms/step - loss: 0.6940 - accuracy: 0.5189 - val_loss: 0.6925 - val_accuracy

**ResNet**

Model

In [10]:
from tensorflow.keras.layers import Input, Add, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

def resnet_block(input_tensor, filters, kernel_size=3):
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same', activation='relu')(input_tensor)
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = Add()([x, input_tensor])
    return x

def create_resnet_model(input_shape=(150, 150, 1)):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = resnet_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = resnet_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = resnet_block(x, 256)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

create_resnet_model().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 1)]        0         []                            
                                                                                                  
 conv2d_60 (Conv2D)          (None, 150, 150, 64)         640       ['input_1[0][0]']             
                                                                                                  
 conv2d_61 (Conv2D)          (None, 150, 150, 64)         36928     ['conv2d_60[0][0]']           
                                                                                                  
 conv2d_62 (Conv2D)          (None, 150, 150, 64)         36928     ['conv2d_61[0][0]']           
                                                                                              

Training

In [11]:
# Train a ResNet-like model for each class
trained_models_resnet = {}

for class_name, (x_train, x_val, y_train, y_val) in data.items():
    print(f"Training ResNet model for class: {class_name}")

    model = create_resnet_model(input_shape=(150, 150, 1))

    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True),
        ModelCheckpoint(f'/content/drive/MyDrive/TFG/kitchen-obj-classification/models/resnet_model_{class_name}.h5', save_best_only=True)
    ]

    # Train the model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=50,
        batch_size=16,
        callbacks=callbacks
    )

    trained_models_resnet[class_name] = model

    print(f"ResNet model for class {class_name} trained and saved.")

Training ResNet model for class: sink
Epoch 1/50
7/7 [==============================] - 29s 4s/step - loss: 2.1121 - accuracy: 0.4623 - val_loss: 2.6120 - val_accuracy: 0.4815
Epoch 2/50
7/7 [==============================] - 9s 1s/step - loss: 1.1682 - accuracy: 0.5472 - val_loss: 0.9560 - val_accuracy: 0.4815
Epoch 3/50
7/7 [==============================] - 4s 593ms/step - loss: 1.0977 - accuracy: 0.4528 - val_loss: 0.7412 - val_accuracy: 0.5185
Epoch 4/50
7/7 [==============================] - 5s 796ms/step - loss: 0.7543 - accuracy: 0.5377 - val_loss: 0.6996 - val_accuracy: 0.4444
Epoch 5/50
7/7 [==============================] - 6s 922ms/step - loss: 0.7005 - accuracy: 0.5189 - val_loss: 0.6938 - val_accuracy: 0.5556
Epoch 6/50
7/7 [==============================] - 10s 2s/step - loss: 0.7155 - accuracy: 0.5377 - val_loss: 0.6931 - val_accuracy: 0.5185
Epoch 7/50
7/7 [==============================] - 1s 145ms/step - loss: 0.6891 - accuracy: 0.5943 - val_loss: 0.7286 - val_accura